In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import itertools

alert = pd.read_csv("data/alert_id.csv", header=0, encoding='utf-8')
event = pd.read_csv("data/event_history.csv", header=0, encoding='utf-8')

alert_count = {}
data_set = {}
for evn, message_id in zip(event["event_id"], event["id"]):
    message_ids = message_id.split(',')
    for id in message_ids:
        count = alert_count.get(int(id), 0)
        alert_count.setdefault(int(id), 0)
        alert_count[int(id)] = count + 1
    data_set[evn] = message_ids

#获取top10的alert
alert_count = sorted(alert_count.items(), key=lambda d : d[1], reverse=True)

print "排在前面10位的alert_message,及其统计频数："
print alert_count[:10]

排在前面10位的alert_message,及其统计频数：
[(2, 9101), (32, 5739), (42, 5737), (1, 5702), (46, 5693), (25, 5690), (20, 5681), (45, 5660), (44, 5648), (40, 5591)]


In [ ]:
#对message做聚类(频繁模式挖掘)
class Apriori:
    def __init__(self,min_sup=0.2,dataDic={}):
        self.data = dataDic
        self.size = len(dataDic) #Get the number of events
        self.min_sup = min_sup
        self.min_sup_val = min_sup * self.size #这个数值是频繁算法的分子整数部分
    def find_frequent_1_itemsets(self):
        FreqDic = {} #{itemset1:freq1,itemsets2:freq2}  不管频繁度如何，先计算频繁度，然后装到这里面
        for event in self.data: #event表示字典里的键值，比如T100
            for item in self.data[event]:
                if item in FreqDic: #该成员是否出现过，出没出现过都要对它进行赋值
                    FreqDic[item] += 1 #出现过则+1
                else:
                    FreqDic[item] = 1 #没出现过则初始值为1
        L1 = []
        for itemset in FreqDic: #遍历第一轮“频繁”的成员
            if itemset >= self.min_sup_val: #如果频繁度大于设定值
                L1.append([itemset]) #将该成员推入列表,仅仅是将成员推入列表，频度不管
        return L1

    def has_infrequent_subset(self,c,L_last,k):
        subsets = list(itertools.combinations(c,k-1)) #return list of tuples of items
        for each in subsets:
            each = list(each) #change tuple into list
            if each not in L_last:
                return True
        return False

    def apriori_gen(self,L_last): #L_last means frequent(k-1) itemsets 难点！！！
        k = len(L_last[0]) + 1 #len()对于列表，则返回成员的个数 如['I1','I2']就返回2
        Ck = [] #候选项集集合
        for itemset1 in L_last: #循环遍历L_last中的成员
            for itemset2 in L_last:
                #join step
                flag = 0
                for i in range(k-2):
                    if itemset1[i] != itemset2[i]:
                        flag = 1 #the two itemset can't join
                        break
                if flag == 1:
                    continue
                if itemset1[k-2] < itemset2[k-2]:
                    c = itemset1 + [itemset2[k-2]] #例：['I1']<['I2']成立，则推入一个['I1','I2']，并且先不计算它的频繁度
                else:
                    continue
                     #pruning setp
                if self.has_infrequent_subset(c,L_last,k):
                    continue
                else:
                    Ck.append(c) #候选项集集合,还没有计算频繁度，先把各种组合扔进去
        return Ck

    def do(self):
        L_last = self.find_frequent_1_itemsets()
        L = L_last
        i = 0
        while L_last != []:
            Ck = self.apriori_gen(L_last)
            FreqDic = {}
            for event in self.data:
                #get all suported subsets
                for c in Ck:
                    if set(c) <= set(self.data[event]):#is subset
                        if tuple(c) in FreqDic:
                            FreqDic[tuple(c)]+=1
                        else:
                            FreqDic[tuple(c)]=1
        return FreqDic
a=Apriori(dataDic=data_set)
mess_dict = a.do()
mess_dict = sorted(mess_dict.items(), key=lambda d : d[1], reverse=True) 
print mess_dict[:10]

In [ ]:
alert = pd.read_csv("data/alert_id.csv", header=0, encoding='utf-8')
event = pd.read_csv("data/event_history.csv", header=0, encoding='utf-8')

alert_count = {}
data_set = []
for message_id in event['id']:
    message_ids = message_id.split(',')
    for id in message_ids:
        count = alert_count.get(int(id), 0)
        alert_count.setdefault(int(id), 0)
        alert_count[int(id)] = count + 1
    data_set.append(message_ids)

#其它方法
import pyfpgrowth
print pyfpgrowth.find_frequent_patterns(data_set, 5)